In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%run ../load_magic/storage.py
%run ../load_magic/lists.py
from PIL import Image
import numpy as np
from IPython.display import HTML
from pathlib import Path
import traceback
import webbrowser
import itertools
import operator

# TEXTURES_DIR = '../data/1.15_Default_Resource_Pack/assets/minecraft/textures/block'
TEXTURES_DIR = '../data/1.18.1_Default_Resource_Pack/assets/minecraft/textures/block'
!start %windir%\explorer.exe "{os.path.abspath(TEXTURES_DIR)}"
s = Storage()


----

In [3]:

b = s.pickle_exists('AVERAGE_DICT') and s.pickle_exists('DOMINANT_DICT')
if b and s.pickle_exists('WEIGHTED_AVERAGE_DICT'):
    AVERAGE_DICT = s.load_object('AVERAGE_DICT')
    DOMINANT_DICT = s.load_object('DOMINANT_DICT')
    WEIGHTED_AVERAGE_DICT = s.load_object('WEIGHTED_AVERAGE_DICT')
else:
    def get_dictionaries(textures_dir, n_colors=5):
        average_dict = {}
        dominant_dict = {}
        weighted_dict = {}
        for file_name in os.listdir(textures_dir):
            if file_name.endswith('.png'):

                # Read the image
                file_path = os.path.join(textures_dir, file_name)
                try:
                    img_array = io.imread(file_path)[:, :, :3]
                    if img_array.shape == (16, 16, 3):

                        # Calculate the mean of each chromatic channel
                        average = img_array.mean(axis=0).mean(axis=0)
                        average_dict[file_name] = tuple(average)

                        # Get the palette color which occurs most frequently
                        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
                        flags = cv2.KMEANS_RANDOM_CENTERS
                        pixels = np.float32(img_array.reshape(-1, 3))
                        _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
                        _, counts = np.unique(labels, return_counts=True)
                        dominant = palette[np.argmax(counts)]
                        dominant_dict[file_name] = tuple(dominant)

                        # Calculate the mean of the palette patch
                        indices = np.argsort(counts)[::-1]   
                        freqs = np.cumsum(np.hstack([[0], counts[indices]/float(counts.sum())]))
                        rows = np.int_(img_array.shape[0]*freqs)
                        palette_patch = np.zeros(shape=img_array.shape, dtype=np.uint8)
                        for i in range(len(rows) - 1):
                            palette_patch[rows[i]:rows[i + 1], :, :] += np.uint8(palette[indices[i]])
                        weighted_average = palette_patch.mean(axis=0).mean(axis=0)
                        weighted_dict[file_name] = tuple(weighted_average)
                except IndexError as e:
                    print(f'{file_name}: {str(e).strip()}')

        return average_dict, dominant_dict, weighted_dict
    AVERAGE_DICT, DOMINANT_DICT, WEIGHTED_AVERAGE_DICT = get_dictionaries(TEXTURES_DIR, n_colors=5)
    s.store_objects(AVERAGE_DICT=AVERAGE_DICT, DOMINANT_DICT=DOMINANT_DICT,
                    WEIGHTED_AVERAGE_DICT=WEIGHTED_AVERAGE_DICT)

In [4]:

pickle_name = 'minecraft_glass_and_concrete_and_terracotta_list'
if s.pickle_exists(pickle_name):
    glass_and_concrete_and_terracotta_list = s.load_object(pickle_name)
else:
    glass_and_concrete_and_terracotta_list = glass_list + concrete_list
    l = [key for key in DOMINANT_DICT.keys() if 'terracotta' in key.lower()]
    glass_and_concrete_and_terracotta_list += l
    s.store_objects(**{pickle_name: glass_and_concrete_and_terracotta_list})
if s.pickle_exists('minecraft_glass_and_terracotta_list'):
    glass_and_terracotta_list = s.load_object('minecraft_glass_and_terracotta_list')
else:
    l = [key for key in DOMINANT_DICT.keys() if 'terracotta' in key.lower()]
    glass_and_terracotta_list = glass_list + l
    s.store_objects(minecraft_glass_and_terracotta_list=glass_and_terracotta_list)
if s.pickle_exists('minecraft_terracotta_list'):
    terracotta_list = s.load_object('minecraft_terracotta_list')
else:
    terracotta_list = [key for key in DOMINANT_DICT.keys() if 'terracotta' in key.lower()]
    s.store_objects(minecraft_terracotta_list=terracotta_list)

In [5]:

b = s.pickle_exists('minecraft_glass_and_concrete_list')
b = b and s.pickle_exists('minecraft_concrete_list')
if b and s.pickle_exists('minecraft_glass_list'):
    glass_and_concrete_list = s.load_object('minecraft_glass_and_concrete_list')
    concrete_list = s.load_object('minecraft_concrete_list')
    glass_list = s.load_object('minecraft_glass_list')
else:
    glass_and_concrete_list = []
    concrete_list = []
    for file_name in AVERAGE_DICT.keys():
        if ('concrete' in file_name):
            (img_array, avg_patch, dom_patch, palette_patch,
             weighted_avg_patch) = get_patches(TEXTURES_DIR, file_name, AVERAGE_DICT,
                                               DOMINANT_DICT, WEIGHTED_AVERAGE_DICT)
            show_images(file_name, img_array, avg_patch, dom_patch, palette_patch,
                        weighted_avg_patch)
            glass_and_concrete_list.append(file_name)
            concrete_list.append(file_name)
    glass_list = []
    for file_name in AVERAGE_DICT.keys():
        if ('glass' in file_name) and ('pane' not in file_name):
            (img_array, avg_patch, dom_patch, palette_patch,
             weighted_avg_patch) = get_patches(TEXTURES_DIR, file_name, AVERAGE_DICT,
                                               DOMINANT_DICT, WEIGHTED_AVERAGE_DICT)
            show_images(file_name, img_array, avg_patch, dom_patch, palette_patch,
                        weighted_avg_patch)
            glass_and_concrete_list.append(file_name)
            glass_list.append(file_name)
    s.store_objects(minecraft_glass_and_concrete_list=glass_and_concrete_list,
                    minecraft_concrete_list=concrete_list, minecraft_glass_list=glass_list)

In [6]:

pickle_name = 'minecraft_glass_and_concrete_and_unglazed_terracotta_list'
if s.pickle_exists(pickle_name):
    glass_and_concrete_and_unglazed_terracotta_list = s.load_object(pickle_name)
else:
    glass_and_concrete_and_unglazed_terracotta_list = []
    for fn in glass_and_concrete_and_terracotta_list:
        if 'glazed' not in fn:
            glass_and_concrete_and_unglazed_terracotta_list.append(fn)
    s.store_objects(**{pickle_name: glass_and_concrete_and_unglazed_terracotta_list})

In [7]:

pickle_name = 'minecraft_concrete_and_unglazed_terracotta_list'
if s.pickle_exists(pickle_name):
    concrete_and_unglazed_terracotta_list = s.load_object(pickle_name)
else:
    for fn in concrete_list + terracotta_list:
        if 'glazed' not in fn:
            concrete_and_unglazed_terracotta_list.append(fn)
    s.store_objects(**{pickle_name: concrete_and_unglazed_terracotta_list})

In [8]:

if s.pickle_exists('minecraft_wool_list'):
    wool_list = s.load_object('minecraft_wool_list')
else:
    wool_list = [fn for fn in blocks_list if 'wool' in fn]
    s.store_objects(minecraft_wool_list=wool_list)

In [9]:

if s.pickle_exists('minecraft_blocks_list'):
    blocks_list = s.load_object('minecraft_blocks_list')
else:
    blocks_list = ['acacia_log.png', 'acacia_log_top.png', 'acacia_planks.png',
                   'andesite.png', 'birch_log.png', 'birch_log_top.png',
                   'birch_planks.png', 'black_concrete.png', 'black_concrete_powder.png',
                   'black_glazed_terracotta.png', 'black_terracotta.png', 'black_wool.png',
                   'blue_concrete.png', 'blue_concrete_powder.png', 'blue_glazed_terracotta.png',
                   'blue_terracotta.png', 'blue_wool.png', 'bone_block_side.png',
                   'bone_block_top.png', 'bookshelf.png', 'bricks.png',
                   'brown_concrete.png', 'brown_concrete_powder.png', 'brown_glazed_terracotta.png',
                   'brown_terracotta.png', 'brown_wool.png', 'chiseled_nether_bricks.png',
                   'chiseled_quartz_block.png', 'chiseled_quartz_block_top.png',
                   'chiseled_red_sandstone.png',
                   'chiseled_sandstone.png', 'chiseled_stone_bricks.png', 'coal_block.png',
                   'coal_ore.png', 'cobblestone.png', 'cracked_nether_bricks.png',
                   'cracked_stone_bricks.png', 'cyan_concrete.png', 'cyan_concrete_powder.png',
                   'cyan_glazed_terracotta.png', 'cyan_terracotta.png', 'cyan_wool.png',
                   'dark_oak_log.png', 'dark_oak_log_top.png', 'dark_oak_planks.png',
                   'dark_prismarine.png', 'diamond_block.png', 'diorite.png',
                   'dirt.png', 'emerald_block.png', 'end_stone.png',
                   'end_stone_bricks.png', 'furnace_front.png', 'furnace_side.png',
                   'furnace_top.png', 'granite.png', 'grass_block_side.png',
                   'gravel.png', 'gray_concrete.png', 'gray_concrete_powder.png',
                   'gray_glazed_terracotta.png', 'gray_terracotta.png', 'gray_wool.png',
                   'green_concrete.png', 'green_concrete_powder.png',
                   'green_glazed_terracotta.png',
                   'green_terracotta.png', 'green_wool.png', 'hay_block_side.png',
                   'hay_block_top.png', 'iron_block.png', 'iron_ore.png',
                   'jack_o_lantern.png', 'jungle_log.png', 'jungle_log_top.png',
                   'jungle_planks.png', 'lapis_block.png', 'light_blue_concrete.png',
                   'light_blue_concrete_powder.png', 'light_blue_glazed_terracotta.png',
                   'light_blue_terracotta.png',
                   'light_blue_wool.png', 'light_gray_concrete.png',
                   'light_gray_concrete_powder.png',
                   'light_gray_glazed_terracotta.png', 'light_gray_terracotta.png',
                   'light_gray_wool.png',
                   'lime_concrete.png', 'lime_concrete_powder.png', 'lime_glazed_terracotta.png',
                   'lime_terracotta.png', 'lime_wool.png', 'magenta_concrete.png',
                   'magenta_concrete_powder.png', 'magenta_glazed_terracotta.png',
                   'magenta_terracotta.png',
                   'magenta_wool.png', 'melon_side.png', 'melon_top.png',
                   'mossy_cobblestone.png', 'mossy_stone_bricks.png', 'netherite_block.png',
                   'netherrack.png', 'nether_bricks.png', 'nether_quartz_ore.png',
                   'nether_wart_block.png', 'note_block.png', 'oak_log.png',
                   'oak_log_top.png', 'oak_planks.png', 'orange_concrete.png',
                   'orange_concrete_powder.png', 'orange_glazed_terracotta.png',
                   'orange_terracotta.png',
                   'orange_wool.png', 'packed_ice.png', 'pink_concrete.png',
                   'pink_concrete_powder.png', 'pink_glazed_terracotta.png', 'pink_terracotta.png',
                   'pink_wool.png', 'piston_side.png', 'piston_top.png',
                   'piston_top_sticky.png', 'podzol_side.png', 'polished_andesite.png',
                   'polished_diorite.png', 'polished_granite.png', 'prismarine_bricks.png',
                   'pumpkin_side.png', 'pumpkin_top.png', 'purple_concrete.png',
                   'purple_concrete_powder.png', 'purple_glazed_terracotta.png',
                   'purple_terracotta.png',
                   'purple_wool.png', 'purpur_block.png', 'purpur_pillar.png',
                   'purpur_pillar_top.png', 'quartz_block_side.png', 'quartz_block_top.png',
                   'quartz_bricks.png', 'quartz_pillar.png', 'quartz_pillar_top.png',
                   'redstone_block.png', 'redstone_lamp.png', 'redstone_ore.png',
                   'red_concrete.png', 'red_concrete_powder.png', 'red_glazed_terracotta.png',
                   'red_nether_bricks.png', 'red_sand.png', 'red_sandstone.png',
                   'red_sandstone_top.png', 'red_terracotta.png', 'red_wool.png',
                   'sand.png', 'sandstone.png', 'sandstone_top.png',
                   'slime_block.png', 'smooth_stone.png', 'smooth_stone_slab_side.png',
                   'soul_sand.png', 'sponge.png', 'spruce_log.png',
                   'spruce_log_top.png', 'spruce_planks.png', 'stone.png',
                   'stone_bricks.png', 'terracotta.png', 'wet_sponge.png',
                   'white_concrete.png', 'white_concrete_powder.png', 'white_glazed_terracotta.png',
                   'white_terracotta.png', 'white_wool.png', 'yellow_concrete.png',
                   'yellow_concrete_powder.png', 'yellow_glazed_terracotta.png',
                   'yellow_terracotta.png', 'yellow_wool.png']
    s.store_objects(minecraft_blocks_list=blocks_list)


----

In [10]:

def rgb_to_hex(rgb):
    
    return '%02x%02x%02x' % rgb

In [11]:

def collate(blocks_list):
    it = itertools.groupby(blocks_list, operator.itemgetter(1))
    for key, subiter in it:
        
        yield key, list(item[0] for item in subiter)

In [27]:

from matplotlib.colors import to_hex

def get_column_markup(img_array, rgb_dict, td_style, img_style, row, col, text_html_str,
                      image_html_str, rows_list=[]):
    f = lambda item: np.linalg.norm(np.array(item[1])-img_array[row][col])
    file_name = sorted(rgb_dict.items(), key=f)[0][0]
    img_path = os.path.abspath(f'{TEXTURES_DIR}/{file_name}')
    block_name = file_name.split('.')[0].replace('_', ' ').title()
    hex_str = to_hex(list(map(lambda x: x/255, rgb_dict[file_name])))
    text_html_str += f'<td style="background-color:{hex_str};text-align:center;">{block_name}</td>'
    image_html_str += f'<td title="{block_name}" style="{td_style}">'
    image_html_str += f'<img src="file:///{img_path}" style="{img_style}" /></td>'
    row_dict = {}
    row_dict['row_number'] = row
    row_dict['column_number'] = col
    row_dict['block_name'] = block_name
    rows_list.append(row_dict)
    
    return rows_list, text_html_str, image_html_str

In [13]:

def get_rowspan_markup(file_prefix, file_path, td_style, image_html_str, row_count, col_count):
    title = file_prefix.replace('_', ' ').title()
    src_url = 'file:///' + os.path.abspath(file_path).replace(os.sep, '/')
    # image_str = f'<img src="{src_url}" style="display:block;" width="100%" height="100%" />'
    image_td_style = f"{td_style}background-image:url('{src_url}');background-size:cover;"
    image_td_style += 'background-position:center;'
    image_html_str += f'<td title="{title}" style="{image_td_style}" rowspan={row_count} '
    image_html_str += f'width="{col_count*16}px"></td>'
    
    return image_html_str

In [14]:

def get_stack_summary(block_count, block_name, stacks_list):
    stack_count = (block_count // 64) + 1
    row_tuple = (block_name, stack_count)
    stacks_list.append(row_tuple)
    stack_str = 's' if stack_count > 1 else ''
    line_count = (block_count // (64*9)) + 1
    line_str = 's' if line_count > 1 else ''
    chest_count = (block_count // (64*9*3)) + 1
    chest_str = 's' if chest_count > 1 else ''
    it_str = 'them' if chest_count > 1 else 'it'
    summary_str = f'{block_name}: {block_count:,} '
    summary_str += f'({chest_count} chest{chest_str} with {line_count} line{line_str}/{stack_count} '
    summary_str += f'stack{stack_str} in {it_str}, total)'
    
    return stacks_list, summary_str

In [15]:

def get_stack_collation(blocks_list, stack_count):
    if len(blocks_list) > 1:
        each_str = 'each '
    else:
        each_str = ''
    if stack_count > 1:
        s_str = 's'
    else:
        s_str = ''
    stack_collation = f'{stack_count} stack{s_str} {each_str}of {conjunctify_list(blocks_list)}.'
    
    return stack_collation

In [16]:

def get_row_markup(text_html_str, image_html_str, rows_list, img_array, rgb_dict, td_style,
                   img_style, file_prefix, file_path,
                   col_start=0, col_count=10, row_start=0, row_count=10):
    text_html_str += '<tr>'
    image_html_str += '<tr>'
    for col in range(col_start, col_count):
        (rows_list, text_html_str,
         image_html_str) = get_column_markup(img_array, rgb_dict, td_style, img_style, row_start,
                                             col, text_html_str, image_html_str, rows_list)
    text_html_str += '</tr>'
    if row_start==0:
        image_html_str = get_rowspan_markup(file_prefix, file_path, td_style, image_html_str,
                                            row_count, col_count)
    image_html_str += '</tr>'
    
    return text_html_str, image_html_str, rows_list

In [26]:

def get_it_markup(file_path, rgb_dict, file_prefix, rows_list=[]):
    minecraft_pixel_art_img = Image.open(file_path)
    img_array = np.array(minecraft_pixel_art_img)
    row_count = img_array.shape[0]
    col_count = img_array.shape[1]
    td_style = 'padding:0;margin:0;'
    img_style = 'display:block;margin:0!important;padding:0!important;border:0!important;'
    text_html_str = image_html_str = '<table style="border-collapse:collapse;">'
    for row in range(row_count):
        (text_html_str, image_html_str,
         rows_list) = get_row_markup(text_html_str, image_html_str, rows_list, img_array, rgb_dict,
                                     td_style, img_style, file_prefix, file_path,
                                     col_start=0, col_count=col_count,
                                     row_start=row, row_count=row_count)
    text_html_str += '</table><hr />'
    image_html_str += '</table>'
    
    return text_html_str, image_html_str, rows_list

In [18]:

def partition(lst, n):
    division = len(lst) / n
    
    return [lst[round(division * i):round(division * (i + 1))] for i in range(n)]

def get_multiples(block_count, by_multiple=10):
    
    return [(r.start, r.stop) for r in partition(range(block_count), block_count // by_multiple)]

In [19]:

def get_it_markup_by_tens(file_path, rgb_dict, file_prefix):
    minecraft_pixel_art_img = Image.open(file_path)
    img_array = np.array(minecraft_pixel_art_img)
    row_count = img_array.shape[0]
    col_count = img_array.shape[1]
    td_style = 'padding:0;margin:0;'
    img_style = 'display:block;margin:0!important;padding:0!important;border:0!important;'
    tuples_list = []
    for row_tuple in get_multiples(row_count):
        for col_tuple in get_multiples(col_count):
            text_html_str = image_html_str = '<table style="border-collapse:collapse;">'
            for row in range(**row_tuple):
                (text_html_str, image_html_str,
                 rows_list) = get_row_markup(text_html_str, image_html_str, rows_list, img_array,
                                             rgb_dict, td_style, img_style, file_prefix, file_path,
                                             col_start=col_tuple[0], col_count=col_tuple[1],
                                             row_start=row, row_count=row_tuple[1]-row_tuple[0]+1)
            text_html_str += '</table><hr />'
            image_html_str += '</table>'
            tuples_list.append(text_html_str, image_html_str)
    
    return tuples_list

In [20]:

def show_art_recipe(file_path, rgb_dict=WEIGHTED_AVERAGE_DICT, blocks_list=None):
    if blocks_list is not None:
        rgb_dict = {k: v for k, v in rgb_dict.items() if k in blocks_list}
    file_prefix = file_path.split('/')[-1].split('.')[0]
    text_html_str, image_html_str, rows_list = get_it_markup(file_path, rgb_dict, file_prefix)
    display(HTML(text_html_str))
    html_path = os.path.abspath(f'../saves/html/{file_prefix}.html')
    os.makedirs(name=os.path.dirname(html_path), exist_ok=True)
    Path(html_path).touch()
    _, _, _, code = traceback.extract_stack()[-2]
    with open(html_path, 'w') as f:
        f.write(f'<html><head><title>{code}</title></head><body>')
        f.write(image_html_str)
        f.write('</body></html>')
    block_names_df = pd.DataFrame(rows_list)
    if blocks_list is not None:
        mask_series = block_names_df.block_name.isin([file_name.split('.')[0].replace('_', ' ').title() for file_name in blocks_list])
        block_names_df = block_names_df[mask_series]
    block_names_series = block_names_df.block_name.value_counts()
    stacks_list = []
    block_names_list = []
    for block_name, block_count in block_names_series.iteritems():
        stacks_list, summary_str = get_stack_summary(block_count, block_name, stacks_list)
        print(summary_str)
        block_names_list.append(block_name)
    print()
    for stack_count, blocks_list in collate(stacks_list):
        stack_collation = get_stack_collation(blocks_list, stack_count)
        print(stack_collation)
    webbrowser.open(html_path, new=2)
    
    return block_names_df


----

In [28]:

file_path = '../saves/png/visual_construction101x101.png'
blocks_list = [fn for fn in concrete_and_unglazed_terracotta_list+wool_list if 'powder' not in fn.lower()]
blocks_list += [key for key in WEIGHTED_AVERAGE_DICT.keys() if key.endswith('_stained_glass.png')]
block_names_df = show_art_recipe(file_path, rgb_dict=DOMINANT_DICT, blocks_list=blocks_list)

Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,White Wool,White Wool,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass
Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,White Concrete,White Terracotta,White Terracotta,White Terracotta,White Concrete,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,White Concrete,White Terracotta,White Terracotta,White Terracotta,White Concrete,White Concrete,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass
Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,Lime Stained Glass,L

Lime Stained Glass: 3,282 (2 chests with 6 lines/52 stacks in them, total)
Brown Wool: 934 (1 chest with 2 lines/15 stacks in it, total)
Light Gray Terracotta: 822 (1 chest with 2 lines/13 stacks in it, total)
White Terracotta: 695 (1 chest with 2 lines/11 stacks in it, total)
Brown Terracotta: 630 (1 chest with 2 lines/10 stacks in it, total)
Terracotta: 480 (1 chest with 1 line/8 stacks in it, total)
Orange Stained Glass: 402 (1 chest with 1 line/7 stacks in it, total)
Brown Concrete: 376 (1 chest with 1 line/6 stacks in it, total)
Yellow Terracotta: 339 (1 chest with 1 line/6 stacks in it, total)
Gray Terracotta: 320 (1 chest with 1 line/6 stacks in it, total)
Brown Stained Glass: 270 (1 chest with 1 line/5 stacks in it, total)
Light Gray Wool: 270 (1 chest with 1 line/5 stacks in it, total)
White Concrete: 224 (1 chest with 1 line/4 stacks in it, total)
Green Terracotta: 224 (1 chest with 1 line/4 stacks in it, total)
Black Terracotta: 208 (1 chest with 1 line/4 stacks in it, total


----

In [ ]:

def get_file_names_dataframe(file_path, blocks_list, rgb_dict):
    art_img = Image.open(file_path)
    img_array = np.array(art_img)
    row_count = img_array.shape[0]
    col_count = img_array.shape[1]
    rows_list = []
    for row in range(row_count):
        for col in range(col_count):
            f = lambda item: np.linalg.norm(np.array(item[1])-img_array[row][col])
            file_name = sorted(rgb_dict.items(), key=f)[0][0]
            row_dict = {}
            row_dict['row_number'] = row
            row_dict['column_number'] = col
            row_dict['file_name'] = file_name
            rows_list.append(row_dict)
    file_names_df = pd.DataFrame(rows_list)
    
    return file_names_df

In [ ]:

file_path = '../saves/png/visual_construction101x101.png'
blocks_list = [fn for fn in concrete_and_unglazed_terracotta_list+wool_list if 'powder' not in fn.lower()]
blocks_list += [key for key in WEIGHTED_AVERAGE_DICT.keys() if key.endswith('_stained_glass.png')]
rgb_dict = {k: v for k, v in DOMINANT_DICT.items() if k in blocks_list}
file_names_df = get_file_names_dataframe(file_path, blocks_list, rgb_dict)

In [ ]:

file_names_df

In [ ]:

mask_series = file_names_df.file_name.isin(['cyan_terracotta.png'])
mean_row_number = file_names_df[mask_series].row_number.mean()
mean_column_number = file_names_df[mask_series].column_number.mean()
file_names_df['middleness'] = file_names_df.row_number.map(lambda x: abs(x - mean_row_number))
file_names_df['centerness'] = file_names_df.row_number.map(lambda x: abs(x - mean_row_number))
file_names_df[mask_series].sort_values(['middleness', 'centerness'])

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

M = 100
angle = np.exp(1j * 2 * np.pi / M)
angles = np.cumprod(np.ones(M + 1) * angle)
x, y = np.real(angles), np.imag(angles)
X = Y = 51
r = 11
edge_set = set()
for x_y_tuple in zip([int(point_x) for point_x in X + r * x], [int(point_y) for point_y in Y + r * y]):
    edge_set.add(x_y_tuple)

In [ ]:

import math

diameter = 21
radius = diameter / 2
if (diameter % 2) == 0:
    maxblocks = math.ceil(radius - .5) * 2 + 1
else:
    maxblocks = math.ceil(radius) * 2
for col in range(int(-maxblocks / 2) + 1, int(maxblocks / 2)):
    print(col)

In [ ]:

def distance(first_tuple, second_tuple):
    first_row, first_colum = first_tuple
    second_row, second_colum = second_tuple
    row_distance = abs(first_row - second_row)
    column_distance = abs(first_colum - second_colum)
    
    return math.sqrt(row_distance**2 + column_distance**2)

In [ ]:

def get_filename(row, col):
    mask_series = (file_names_df.row_number == row) & (file_names_df.column_number == col)
    file_name = file_names_df[mask_series].file_name.squeeze()
    
    return file_name

In [ ]:

def get_circle_html(middle_row, center_column, diameter):
    radius = diameter / 2
    if (diameter % 2) == 0:
        maxblocks = math.ceil(radius - .5) * 2 + 1
    else:
        maxblocks = math.ceil(radius) * 2
    west_column = north_row = middle_center - diameter
    east_column = south_row = middle_center + diameter
    td_style = 'padding:0;margin:0;'
    img_style = 'display:block;margin:0!important;padding:0!important;border:0!important;'
    html_str = '<table style="border-collapse:collapse;">'
    for row in range(int(-maxblocks / 2) + 1, int(maxblocks / 2)):
        html_str += '<tr>'
        for col in range(int(-maxblocks / 2) + 1, int(maxblocks / 2)):
            file_name = get_filename(middle_row+row, center_column+col)
            block_name = file_name.split('.')[0].replace('_', ' ').title()
            html_str += f'<td title="{block_name}" style="{td_style}">'
            if distance((middle_row, center_column), (middle_row+row, center_column+col)) <= radius:
                img_path = os.path.abspath(os.path.join(TEXTURES_DIR, file_name))
                html_str += f'<img src="file:///{img_path}" style="{img_style}" /></td>'
            else:
                html_str += '</td>'
        html_str += '</tr>'
    html_str += '</table>'
    
    return html_str

In [ ]:

middle_center = 51
diameter = 21
html_str = get_circle_html(middle_center, middle_center, diameter)
html_path = os.path.abspath(f'../saves/html/tens.html')
os.makedirs(name=os.path.dirname(html_path), exist_ok=True)
Path(html_path).touch()
with open(html_path, 'w') as f:
    f.write(f'<html><head><title>Tens</title></head><body>')
    f.write(html_str)
    f.write('</body></html>')
webbrowser.open(html_path, new=2)